# Galaxy and Non-Galaxy Classifiers:

Here, we developed five machine learning classifiers for [Galaxy Zoo 2](https://data.galaxyzoo.org/#section-7) (GZ2) cataloge images to recognize the galaxy images from non-galaxy images. These five classification models use the morphological and shape information. We applied the threshold based tasks (features or disk fraction, edge-on no fraction, spiral fraction, smooth fraction, completely round fraction, odd no fraction, and odd yes fraction) to collect 780 galaxy sample images. Also, we used threshold tasks (star or artifact fraction) to collect 545 non-galaxy images. 

- Two classifier models, including support vector machine (SVM) and classic 1D-convolutional neural network (1D-CNN), have been designed to use the Zernike moments (ZMs) extracted from the original galaxy and non-galaxy images. 
- Three classifier models including CNN-Vision Transformer, ResNet50, VGG16 have investigated to work the information of original galaxy and non-galaxy images.

### Import libraries:

The list of requried libraries are sklearn, pandas, numpy, tensorflow, matplotlib, etc. 

In [ ]:
#Import packages

import os
from sklearn.svm import SVC
from sklearn import metrics
import pickle
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import (Dense, Dropout,BatchNormalization, Input, Conv1D, Flatten,
                             MaxPooling1D)
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


#Scikit_learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, roc_auc_score, auc, log_loss,
                             precision_score, recall_score, f1_score,
                             accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)

## Two classifier models based on Zernike moments (ZMs):

### Compute the ZMs:

##### First we need to compute ZMs for galaxy and non-galaxy images. The ZEMO python package [https://pypi.org/project/ZEMO/] [https://github.com/hmddev1/ZEMO] can be used to compute Zernike moments (ZMs) for images. This package was described in the research paper [[IAJJ](https://ijaa.du.ac.ir/article_374_ad45803d737b0a7d4fc554a244229df6.pdf)].

*Note: The galaxy and non-galaxy images are in RGB format. Here, we used the R channel of images. The size of original Galaxy Zoo 2 images is (424, 424) pixels, while we resized them to (200, 200) pixels. To compute ZMs we set the maximum order number $P{max} = 45$.* 

In [ ]:
from ZEMO import zemo
import cv2

def calculate_zernike_moments(data_dir, image_size, zernike_order):
        
        ZBFSTR = zemo.zernike_bf(image_size, zernike_order, 1)
        
        image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]
        
        zernike_moments = []
    
        for img_path in image_files:
            image = cv2.imread(img_path)
            resized_image = cv2.resize(image, (image_size,image_size))
            im = resized_image[:, :, 0]
            Z = np.abs(zemo.zernike_mom(np.array(im), ZBFSTR))
            zernike_moments.append(Z)
        
        df = pd.DataFrame(zernike_moments)
    
        return df

- Please download the **Data** files from [this link](https://drive.google.com/file/d/1wxmYQ8qpgaVDuD3kTeBrZlyny0IBA9wn/view?usp=drive_link)

- The directoies of galaxy and non-galaxy images:

        - Galaxy: /repository/Data/galaxy_nongalaxy/image/galaxy
        - Non-Galxy: /repository/Data/galaxy_nongalaxy/image/nongalaxy

In [ ]:
galaxy_path = r'/path/to/repository/Data/galaxy_nongalaxy/image/galaxy' 
nongalaxy_path = r'/path/to/repository/Data/galaxy_nongalaxy/image/nongalaxy' 

# Defult image size and zernike order. 
image_size = 200
zernike_order = 45

galaxy_zm_df = calculate_zernike_moments(galaxy_path, image_size, zernike_order)
galaxy_zm_df.to_csv('/path/to/repository/Data/galaxy_nongalaxy/ZMs/galaxy_zms.csv')

nongalaxy_zm_df = calculate_zernike_moments(nongalaxy_path, image_size, zernike_order)
nongalaxy_zm_df.to_csv('/path/to/repository/Data/galaxy_nongalaxy/ZMs/nongalaxy_zms.csv')
np.shape(galaxy_zm_df), np.shape(nongalaxy_zm_df)


*Note: Computing of ZMs for above mentioned galaxy and non-galaxy images are slightly consuming time. So, we upladed the zernike moments of both classes in this repository.*

**To load the ZMs please use:**


In [ ]:
galaxy_zm = pd.read_csv('/path/to/repository/Data/galaxy_nongalaxy/ZMs/galaxy_zms.csv')
nongalaxy_zm = pd.read_csv('/path/to/repository/Data/galaxy_nongalaxy/ZMs/nongalaxy_zms.csv')

galaxy_zm.drop("Unnamed: 0", axis = 1, inplace = True)
nongalaxy_zm.drop("Unnamed: 0", axis = 1, inplace = True)

zmg = np.array(galaxy_zm)
zmng = np.array(nongalaxy_zm)

all_zm_data = np.concatenate([zmg,zmng])
len(zmg), len(zmng), len(all_zm_data)

- We use **"0"** for galaxy class labels and **"1"** for non-galaxy class labels.  

In [ ]:
galaxies_labels = np.zeros(len(zmg))
nongalaxy_labels = np.ones(len(zmng))
all_labels = np.concatenate([galaxies_labels, nongalaxy_labels])
len(all_labels)

### **SVM + ZMs**

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)


- Since, the galaxy and non-galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_zm_data) / (2*len(zmg)), 1: len(all_zm_data) / (2*len(zmng))}

- The **SVM** model uses radial base kernel (rbf), C = 1.5, and gamma = 'scale' to fit the model on the training set:

In [ ]:
model = SVC(kernel='rbf', probability=True, C=1.5, gamma='scale',class_weight=class_weights)
gz2_training_model = model.fit(X_train, y_train)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(X_test)

con = metrics.confusion_matrix(y_test, y_pred)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
auc = metrics.roc_auc_score(y_test, y_pred)

class_names = ['Galaxy', 'None-Galaxy']

plt.figure(figsize=(8, 6))

plt.plot(fpr, tpr, lw=1.5, label='ROC Curve (AUC = {:.3f})'.format(auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('The ROC Curve')
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)
print("AUC:", auc)

### **1D_CNN + ZMs**

- We use **"0"** for galaxy class labels and **"1"** for non-galaxy class labels.  

In [ ]:
galaxies_labels = np.zeros(len(zmg))
nongalaxy_labels = np.ones(len(zmng))
all_labels = np.concatenate([galaxies_labels, nongalaxy_labels])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=2)

- Since, the galaxy and non-galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_zm_data) / (2*len(zmg)), 1: len(all_zm_data) / (2*len(zmng))}

- Due to one dimentional structure of ZMs, we used one dimentional achitecture of CNN: 

In [ ]:
# input value
x = Input(shape=(all_zm_data.shape[1],1))

#hidden layers
c0 = Conv1D(256, kernel_size=3, strides=2, padding="same")(x)
b0 = BatchNormalization()(c0)
m0 = MaxPooling1D(pool_size=2)(b0)
d0 = Dropout(0.1)(m0)

c1 = Conv1D(128, kernel_size=3, strides=2, padding="same")(d0)
b1 = BatchNormalization()(c1)
m1 = MaxPooling1D(pool_size=2)(b1)
d1 = Dropout(0.1)(m1)

c2 = Conv1D(64, kernel_size=3, strides=2, padding="same")(d1)
b2 = BatchNormalization()(c2)
m2 = MaxPooling1D(pool_size=2)(b2)
d2 = Dropout(0.1)(m2)

f = Flatten()(d2)

# output
de0 = Dense(64, activation='relu')(f)
de1 = Dense(32, activation='relu')(de0)
de2 = Dense(2, activation='softmax')(de1)

model = Model(inputs=x, outputs=de2, name="cnn_zm_45_galaxy_nonegalaxy")
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

- The **1D_CNN** model uses EarlyStopping as callback function, batch size = 64, and number of epochs = 30 to fit the model on the training set:

In [ ]:
# Callback Function
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

b_size = 64
e_num = 30

history = model.fit(
X_train, y_train_encoded,
batch_size=b_size,
epochs=e_num,
class_weight=class_weights,
verbose = 1,
callbacks=es,
validation_split=0.1)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_labels)
auc = metrics.roc_auc_score(y_test, y_pred_labels)

class_names = ['Galaxy', 'None-Galaxy']

plt.figure(figsize=(8, 6))

plt.plot(fpr, tpr, lw=1.5, label='ROC Curve (AUC = {:.3f})'.format(auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('The ROC Curve')
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)
print("AUC:", auc)

## Three classifier models based on the original images:
  
- (Vision Transformer used as data augmentation tool on the Galaxy and Non-Galaxy images.)

### Import the requried libraries:

In [ ]:
# import packages 

import cv2
import os
import numpy as np
import random
from PIL import Image
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

#Tensorflow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Dense, Dropout, Input,Conv2D, Flatten,
                             MaxPooling2D,BatchNormalization)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50, VGG16
from keras.callbacks import EarlyStopping

#Torch
import torch
import torch.nn as nn
from torchvision import transforms

- To read the images of each class and convert to Pillow images we used the following function:

In [ ]:
def load_galaxy_images(data_dir, target_size):
        
        """
        Loads, resizes, and processes all JPG images from the specified directory.

        Parameters:
        data_dir (str): The directory containing the JPG images to be processed.
        target_size (tuple): The target size for resizing the images, specified as (width, height).

        Returns:
        list: A list of PIL Image objects, each representing a resized and processed image.

        The function performs the following steps:
        1. Lists all JPG image files in the specified directory.
        2. Reads each image using OpenCV.
        3. Resizes each image to the specified target size.
        4. Scales the pixel values and converts the image to a format compatible with PIL.
        5. Converts each resized image to a PIL Image object.
        6. Appends each PIL Image object to a list.
        7. Returns the list of PIL Image objects.
        """

        all_images = []

        file_path = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith('.jpg')]

        for img in file_path:
            image = cv2.imread(img)
            resized_images=cv2.resize(image, target_size)
            resized_images = (resized_images * 255).astype(np.uint8)
            pil_images = Image.fromarray(resized_images)
            all_images.append(pil_images)

        return all_images

- Please use these directories from the repository for image data:
        
        - galaxy: /repository/Data/galaxy_nongalaxy/image/galaxy
        - non-galaxy: /repository/Data/galaxy_nongalaxy/image/nongalaxy

In [ ]:
galaxy_path = r'/path/to/repository/Data/galaxy_nongalaxy/image/galaxy' 
nongalaxy_path = r'/path/to/repository/Data/galaxy_nongalaxy/image/nongalaxy' 

image_size = 200

g_img = load_galaxy_images(galaxy_path, target_size=(image_size,image_size))
ng_img = load_galaxy_images(nongalaxy_path, target_size=(image_size,image_size))

all_data = g_img + ng_img
np.shape(all_data)

- We define the **vision transformer** for both training and testing data sets: 

In [ ]:
# transforms for training data
train_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.RandomRotation(90),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0), ratio=(0.99, 1.01)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])


# transforms for test data
test_transform = transforms.Compose([transforms.CenterCrop(image_size),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

- We use **"0"** for galaxy class labels and **"1"** for non-galaxy class labels.  

In [ ]:
galaxy_labels = np.zeros(len(g_img))
nongalaxy_labels = np.ones(len(ng_img))

all_labels = np.concatenate([galaxy_labels, nongalaxy_labels])
len(all_labels)

### **CNN + vision transformer + original images**

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=2)

- We apply the vision transformer for both training and testing samples:

In [ ]:
# Transformer for training data
transformed_X_train=[]
for i in range(len(X_train)):
  transformed_train_images = train_transform(X_train[i])
  new_image = np.transpose(transformed_train_images, (1, 2, 0))
  transformed_X_train.append(new_image)

# Transformer for testing data
transformed_X_test=[]
for j in range(len(X_test)):
  transformed_test_images = test_transform(X_test[j])
  new_images = np.transpose(transformed_test_images, (1, 2, 0))
  transformed_X_test.append(new_images)

- Since, the galaxy and non-galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_data) / (2*len(g_img)), 1: len(all_data) / (2*len(ng_img))}

- The structure of the classic CNN model:

In [ ]:
# input
x = Input(shape=(image_size,image_size,3))

#hidden layers
c0 = Conv2D(256, kernel_size=(3,3), strides=(1,1), padding="same")(x)
b0 = BatchNormalization()(c0)
m0 = MaxPooling2D(pool_size=(2, 2))(b0)
d0 = Dropout(0.1)(m0)

c1 = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding="same")(m0)
b1 = BatchNormalization()(c1)
m1 = MaxPooling2D(pool_size=(2, 2))(b1)
d1 = Dropout(0.1)(m1)

c2 = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="same")(m1)
b2 = BatchNormalization()(c2)
m2 = MaxPooling2D(pool_size=(2, 2))(b2)
d2 = Dropout(0.1)(m2)

f = Flatten()(m2)

# output layers
de0 = Dense(64, activation='relu')(f)
de1 = Dense(32, activation='relu')(de0)
de2 = Dense(2, activation='softmax')(de1)

model = Model(inputs=x, outputs=de2, name="cnn_transformer_galaxy_nonegalaxy")
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam()
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
model.summary()

- The **CNN** model uses EarlyStopping as callback function, batch size = 64, and number of epochs = 30 to fit the model on the training set:

In [ ]:
# Callback Functions
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

b_size = 64
e_num = 30

history = model.fit(
np.array(transformed_X_train), y_train_encoded,
batch_size=b_size,
epochs=e_num,
verbose = 1,
class_weight=class_weights,
callbacks=es,
validation_split=0.1
)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(np.array(transformed_X_test))
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_labels)
auc = metrics.roc_auc_score(y_test, y_pred_labels)

class_names = ['Galaxy', 'None-Galaxy']

plt.figure(figsize=(8, 6))

plt.plot(fpr, tpr, lw=1.5, label='ROC Curve (AUC = {:.3f})'.format(auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('The ROC Curve')
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)
print("AUC:", auc)

### **ResNet50 + vision transformer + original images**

- We use **"0"** for galaxy class labels and **"1"** for non-galaxy class labels.  

In [ ]:
galaxy_labels = np.zeros(len(g_img))
nongalaxy_labels = np.ones(len(ng_img))

all_labels = np.concatenate([galaxy_labels, nongalaxy_labels])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=2)

- We apply the vision transformer for both training and testing samples:

In [ ]:
# Transformer for training data
transformed_X_train=[]
for i in range(len(X_train)):
  transformed_train_images = train_transform(X_train[i])
  new_image = np.transpose(transformed_train_images, (1, 2, 0))
  transformed_X_train.append(new_image)

# Transformer for testing data
transformed_X_test=[]
for j in range(len(X_test)):
  transformed_test_images = test_transform(X_test[j])
  new_images = np.transpose(transformed_test_images, (1, 2, 0))
  transformed_X_test.append(new_images)

- Since, the galaxy and non-galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_data) / (2*len(g_img)), 1: len(all_data) / (2*len(ng_img))}

- The **ResNet50** model uses EarlyStopping as callback function, batch size = 64, and number of epochs = 30 to fit the model on the training set:

In [ ]:
# Defining the pretrained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)  # The custom layers
output = Dense(2, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(
np.array(transformed_X_train), y_train_encoded,
batch_size=b_size,
epochs=e_num,
verbose = 1,
callbacks=es,
class_weight=class_weights,
validation_split=0.1
)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(np.array(transformed_X_test))
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_labels)
auc = metrics.roc_auc_score(y_test, y_pred_labels)

class_names = ['Galaxy', 'None-Galaxy']

plt.figure(figsize=(8, 6))

plt.plot(fpr, tpr, lw=1.5, label='ROC Curve (AUC = {:.3f})'.format(auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('The ROC Curve')
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)
print("AUC:", auc)

### **VGG16 + vision transformer + original images**

- We use **"0"** for galaxy class labels and **"1"** for non-galaxy class labels.  

In [ ]:
galaxy_labels = np.zeros(len(g_img))
nongalaxy_labels = np.ones(len(ng_img))

all_labels = np.concatenate([galaxy_labels, nongalaxy_labels])
len(all_labels)

- We split the data set into 75 percent traning set and 25 percent test set:

In [ ]:
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_data, all_labels, np.arange(len(all_labels)), 
                                                                                 test_size=0.25, shuffle=True, random_state=None)

y_train_encoded = to_categorical(y_train, num_classes=2)

- We apply the vision transformer for both training and testing samples:

In [ ]:
# Transformer for training data
transformed_X_train=[]
for i in range(len(X_train)):
  transformed_train_images = train_transform(X_train[i])
  new_image = np.transpose(transformed_train_images, (1, 2, 0))
  transformed_X_train.append(new_image)

# Transformer for testing data
transformed_X_test=[]
for j in range(len(X_test)):
  transformed_test_images = test_transform(X_test[j])
  new_images = np.transpose(transformed_test_images, (1, 2, 0))
  transformed_X_test.append(new_images)

- Since, the galaxy and non-galaxy classifiers are unbalance class models, so we used the class weight in the program:

In [ ]:
class_weights = {0: len(all_data) / (2*len(g_img)), 1: len(all_data) / (2*len(ng_img))}

- The **VGG16** model uses EarlyStopping as callback function, batch size = 64, and number of epochs = 30 to fit the model on the training set:

In [ ]:
# Defining the pretrained ResNet50
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)  # The custom layers
output = Dense(2, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(
np.array(transformed_X_train), y_train_encoded,
batch_size=b_size,
epochs=e_num,
verbose = 1,
callbacks=es,
class_weight=class_weights,
validation_split=0.1
)

- Now, we apply the test set to examine the classification algorithm. Using the predicted label by the machine on original labels, we compute the elements of the confusion matrix.

In [ ]:
y_pred = model.predict(np.array(transformed_X_test))
y_pred_labels = np.argmax(y_pred, axis=1)

con = metrics.confusion_matrix(y_test, y_pred_labels)
print(con)

- To compare the performace of classifier with the random classifier, we calculate the **reciver operation charecterstic curve (ROC curve)**. The **area under the curve (AUC)** shows the probability of True positive rates of the classifier.

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_labels)
auc = metrics.roc_auc_score(y_test, y_pred_labels)

class_names = ['Galaxy', 'None-Galaxy']

plt.figure(figsize=(8, 6))

plt.plot(fpr, tpr, lw=1.5, label='ROC Curve (AUC = {:.3f})'.format(auc))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('The ROC Curve')
plt.show()

- To measure the performance metrics of classifier, we compute **(Recall, Precision, f1_score, Accuracy, TSS(True Skill Statistic))**

In [ ]:
recall = recall_score(y_test, y_pred, average= 'weighted')
precision = precision_score(y_test, y_pred, average= 'weighted')
f1_score = f1_score(y_test, y_pred, average= 'weighted')
accuracy = accuracy_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tss=(tp/(tp+fn))-(fp/(fp+tn))

In [ ]:
print("Recall:", recall)
print("Precision:", precision)
print("F1_score:", f1_score)
print("Accuracy:", accuracy)
print("TSS:", tss)
print("AUC:", auc)